# Tutorial: Axial pile capacity calculation using Methode De Beer

Axial pile capacity according to Eurocode 7 is calculated using the method by De Beer in Belgian geotechnical practice.

The current software provided by BGGG-GBMS for calculations according to the base resistance method by De Beer is relatively old and doesn't lend itself to automation. A Python library for application of De Beer's method was developed and is benchmarked against the CPTEX method in this example.

## 1. Package imports

A number of Python packages are imported for data manipulation, printing output in the notebook and interactive plotting (Plotly).

In [ ]:
import numpy as np
from IPython.display import HTML
import pandas as pd

In [ ]:
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode()

The actual code for executing the pile calculation according to De Beer is located in the package ```debeer```.

In [ ]:
from groundhog.deepfoundations.axialcapacity.debeer import DeBeerCalculation
from groundhog.general.soilprofile import SoilProfile

## 2. Data reading

The CPT data can be read from the CPTEX export using the ```read_csv``` function from Pandas. The calculation will be benchmarked against this data.

In [ ]:
df = pd.read_csv("Data/Oud-Turnhout II.csv", delimiter=';', decimal=',')
df.head()

## 3. Calculation setup

A calculation is set up by specifying the depths and corresponding cone tip resistance values as well as the diameter of the pile to be considered. The data is resampled to a 0.2m grid to allow use of De Beer's method using the ```resample_data``` method.

In [ ]:
calc = DeBeerCalculation(
    depth=df['Diepte[m]'],
    qc=df['Conuswaarden[MN/m²]'],
    diameter_pile=0.4)
calc.resample_data()

## 4. Soil layering definition

The soil layering is defined based on the CPT data and requires some engineering interpretation (aided by lab tests or more advanced CPT processing). This is beyond the scope of this tutorial. The method ```set_soil_layers``` requires the definition of the top depths and bottom depths of each layer as well as the soil types. Note that the soil types need to be specific names (see detailed documentation). The water level can also be set. 

In [ ]:
profile = SoilProfile({
    'Depth from [m]': [0, 17.2],
    'Depth to [m]': [17.2, 31],
    'Soil type': ["Sand", "Clayey sand / loam (silt)"],
})
calc.set_soil_layers(
    soilprofile=profile,
    water_level=5)

We can plot the result of the vertical stress calculation:

In [ ]:
fig = calc.layering.plot_profile(
    parameters=(
        (),
        ('Effective vertical stress [kPa]', 'Total vertical stress [kPa]', 'Hydrostatic pressure [kPa]'),),
    showlegends=((), (True, True, True),),
    xtitles=(r'$ q_c \ \text{[MPa]} $',r'$ \sigma_{vo}, \ \sigma_{vo}^{\prime}, \ u_0 \ \text{[kPa]} $',),
    ztitle=r'$ z \ \text{[m]} $',
    xranges=((0, 50), (0, 600),),
    zrange=(31, 0),
    fillcolordict={'Sand': 'yellow', 'Clayey sand / loam (silt)': 'orange', 'SILT': 'green', 'ROCK': 'grey'},
    showfig=False)

qc_data = go.Scatter(x=df['Conuswaarden[MN/m²]'], y=df['Diepte[m]'], showlegend=True, mode='lines',name=r'$ q_c $')
fig.append_trace(qc_data, 1, 2)
fig['layout'].update(
    legend=dict(
        orientation='h',
        x=0.2,
        y=-0.2,
        ))
iplot(fig, filename='plot', config={'showLink': False})

## 5. Base resistance calculation

The entire base resistance calculation algorithm is contained in the method ```calculate_base_resistance```. The documentation provides detailed guidance. The algorithm is encoded based on the original publication by De Beer and the benchmark calculations by MOW geotechniek.

In [ ]:
calc.calculate_base_resistance()

A standardised plot of the base resistance is provided. This plot can be used to draw the comparison between the result from CPTEX and the Python procedure.

In [ ]:
calc.plot_base_resistance(show_fig=False)
trace_cptex = go.Scatter(
    x=df['qb[MN/m²]'], y=df['Diepte[m]'], showlegend=True, mode='lines', name='CPTEX',
    line=dict(width=2, color='black', dash='dot'))
calc.base_plot.append_trace(trace_cptex, 1, 1)
iplot(calc.base_plot, filename='baseplot', config={'showLink': False})

## 6. Shaft resistance calculation

For the shaft resistance calculation, the cone resistance values are first corrected for the cone type effect. These corrections are applicable for mechanical cones in tertiary clay.

In [ ]:
calc.correct_shaft_qc(cone_type='E')

The average cone resistance is subsequenty calculated in each layer:

In [ ]:
calc.calculate_average_qc()

This is followed by a conversion to unit shaft friction according to the Belgian guidance:

In [ ]:
calc.calculate_unit_shaft_friction()

A standardised plot of the averages and the resulting unit shaft friction is provided:

In [ ]:
calc.plot_unit_shaft_friction()

## 7. Calculation of shaft, base and total capacity

The shaft and base capacity can be calculated using the applicable factors. First the $ \alpha_s $ and $ \alpha_b $ factors are set:

In [ ]:
calc.set_shaft_base_factors(
    alpha_b_tertiary_clay=1, alpha_b_other=1,
    alpha_s_tertiary_clay=0.6, alpha_s_other=0.6)

Using this factors, the pile capacity is calculated. Note that the user is still responsible for entering the correct values of the factors.

In [ ]:
calc.calculate_pile_resistance(
    pile_penetration=12, base_area=0.25 * np.pi * (0.51 ** 2),
    circumference=np.pi * 0.51, beta_base=1, lambda_base=1)

The resulting values of pile capacity can be printed to the notebook.

In [ ]:
HTML("""
The shaft capacity $ R_s $ = %.1fkN, the base capacity $ R_b $ = %.1fkN, the total capacity $ R_c $ = %.1fkN""" % (
    calc.Rs, calc.Rb, calc.Rc))